# Simple Economic Conflict Model

This notebook describes a simple, illustrative model for simulating basic economic interactions and conflicts between countries. The code below demonstrates how to build a network of capital cities, produce and exchange resources (like “food”), and introduce conflict by “blocking” nodes.

You can use this framework as a starting point for more sophisticated models involving multiple resources, complex trade rules, and dynamic conflict behaviours.

## Table of Contents

1. [Overview](#overview)  
2. [Key Steps in the Code](#key-steps-in-the-code)  
   1. [Install Packages](#1-install-packages)  
   2. [Imports](#2-imports)  
   3. [Sample Capital Cities Data](#3-sample-capital-cities-data)  
   4. [Distance Function](#4-distance-function)  
   5. [Build the Graph](#5-build-the-graph)  
   6. [Add Edges by Distance](#6-add-edges-by-distance)  
   7. [Simple Resource-Flow Simulation](#7-simple-resource-flow-simulation)  
   8. [Demonstration: Run the Simulation](#8-demonstration-run-the-simulation)  
   9. [Visualization](#9-visualization)  
3. [How the Model Works](#how-the-model-works)  
   1. [Nodes](#nodes)  
   2. [Edges](#edges)  
   3. [Blocked Nodes](#blocked-nodes)  
   4. [Simulation Flow](#simulation-flow)  
4. [How to Extend the Model](#how-to-extend-the-model)  
   1. [Add New Resources](#add-new-resources)  
   2. [Refine Production and Trade Logic](#refine-production-and-trade-logic)  
   3. [Advanced Conflict Rules](#advanced-conflict-rules)  
   4. [Dynamic or Time-Varying Graph Edges](#dynamic-or-time-varying-graph-edges)  
5. [Summary](#summary)


## Overview

This code uses **NetworkX** to model a network of countries’ capital cities. Each city is a node in the graph and has attributes like geographic position, whether it’s “blocked” by conflict, and a resource dictionary (e.g., containing “food”). Edges between cities represent possible trade routes, determined by physical distance. We run a simple simulation in discrete time steps, where cities:
1. Produce a fixed amount of food if they are not blocked.  
2. Trade surplus food with neighboring cities that have a deficit.

When a city is “blocked,” it neither produces nor trades its resources. After running the simulation for a few turns, we visually plot the network on a world map.

## Key Steps in the Code

Below is a step-by-step explanation of the code.

### 1) Install Packages

If using Google Colab or a similar environment, you first need to install the required libraries for geospatial analysis and network modeling:

- **geopandas**  
- **networkx**  
- **shapely**  
- **geopy**  
- **pyproj**  


In [ ]:
!pip install geopandas networkx shapely geopy pyproj

### 2) Imports

We import the following packages:

- **pandas** to handle tabular data for capital cities.  
- **networkx** to create and manipulate the graph.  
- **matplotlib** for plotting.  
- **geopandas** and **shapely** for geospatial operations.  
- **geopy** for calculating great-circle distances.

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd

from shapely.geometry import Point, LineString
from geopy.distance import geodesic

### 3) Sample Capital Cities Data

A small dataset of countries, their capitals, and lat/lon coordinates.

In [ ]:
capitals_data = [
    {"Country": "France",  "Capital": "Paris",    "Latitude": 48.8566,   "Longitude": 2.3522},
    {"Country": "Germany", "Capital": "Berlin",   "Latitude": 52.5200,   "Longitude": 13.4050},
    {"Country": "Spain",   "Capital": "Madrid",   "Latitude": 40.4168,   "Longitude": -3.7038},
    {"Country": "Italy",   "Capital": "Rome",     "Latitude": 41.9028,   "Longitude": 12.4964},
    {"Country": "Egypt",   "Capital": "Cairo",    "Latitude": 30.0444,   "Longitude": 31.2357},
    {"Country": "Kenya",   "Capital": "Nairobi",  "Latitude": -1.286389, "Longitude": 36.817223},
    {"Country": "Japan",   "Capital": "Tokyo",    "Latitude": 35.6895,   "Longitude": 139.6917},
    {"Country": "Brazil",  "Capital": "Brasília", "Latitude": -15.8267,  "Longitude": -47.9218}
]

capitals_df = pd.DataFrame(capitals_data)
capitals_df

### 4) Distance Function

A helper function that returns the distance in kilometers between two lat/lon points, using **geopy**.

In [ ]:
def distance_km(lat1, lon1, lat2, lon2):
    """
    Returns the distance in kilometers between two lat/lon pairs.
    Uses geopy for great-circle distance.
    """
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

### 5) Build the Graph

We create a `NetworkX` undirected graph `G`. Each capital city is added as a node with:
- **pos**: `(longitude, latitude)`
- **country**
- **blocked** (initially False)
- **resources** (100 units of "food")

In [ ]:
G = nx.Graph()

# Add each capital as a node
for idx, row in capitals_df.iterrows():
    capital = row["Capital"]
    lat = row["Latitude"]
    lon = row["Longitude"]
    G.add_node(
        capital,
        pos=(lon, lat),  # (x=lon, y=lat)
        country=row["Country"],
        blocked=False,
        resources={"food": 100}
    )

### 6) Add Edges by Distance

We'll connect capitals if they are within a certain distance threshold (3,000 km). Each edge has:
- **distance**: The geodesic distance in km.
- **capacity**: A placeholder integer (10).

In [ ]:
DISTANCE_THRESHOLD = 3000

all_cities = list(G.nodes())
for i in range(len(all_cities)):
    for j in range(i+1, len(all_cities)):
        cityA = all_cities[i]
        cityB = all_cities[j]
        (lonA, latA) = G.nodes[cityA]["pos"]
        (lonB, latB) = G.nodes[cityB]["pos"]
        
        d_km = distance_km(latA, lonA, latB, lonB)
        
        if d_km <= DISTANCE_THRESHOLD:
            G.add_edge(cityA, cityB, distance=d_km, capacity=10)

### 7) Simple Resource-Flow Simulation

This model simulates a very basic production and trade system:
1. **Produce food**: Each unblocked city produces a fixed amount of food.
2. **Trade food**: Cities with surplus food share it with cities that have a deficit, subject to a threshold and a max trade limit.
3. **Blocked cities**: If a city is blocked, it does not produce or trade.

In [ ]:
def produce_food(G, production_amount=10):
    """
    Each non-blocked city produces a fixed amount of 'food' each turn.
    """
    for city in G.nodes:
        if not G.nodes[city]["blocked"]:
            G.nodes[city]["resources"]["food"] += production_amount

def trade_food(G, trade_threshold=100, max_trade_per_edge=20):
    """
    Simple one-pass trade:
    - If a city has more than 'trade_threshold' food, it tries to share
      the excess with neighbors that have less than 'trade_threshold'.
    - Limited by 'max_trade_per_edge' for demonstration.
    - Blocked cities do not send or receive.
    """
    # Record net flow so we don't double-update resources within the same step
    food_flows = {city: 0 for city in G.nodes}
    
    for cityA, cityB in G.edges():
        if G.nodes[cityA]["blocked"] or G.nodes[cityB]["blocked"]:
            # If either city is blocked, no trade occurs on this edge
            continue
        
        foodA = G.nodes[cityA]["resources"]["food"]
        foodB = G.nodes[cityB]["resources"]["food"]
        
        # City A -> City B
        if foodA > trade_threshold and foodB < trade_threshold:
            surplusA = foodA - trade_threshold
            deficitB = trade_threshold - foodB
            flowAB = min(surplusA, deficitB, max_trade_per_edge)
            food_flows[cityA] -= flowAB
            food_flows[cityB] += flowAB
        
        # City B -> City A
        elif foodB > trade_threshold and foodA < trade_threshold:
            surplusB = foodB - trade_threshold
            deficitA = trade_threshold - foodA
            flowBA = min(surplusB, deficitA, max_trade_per_edge)
            food_flows[cityB] -= flowBA
            food_flows[cityA] += flowBA
    
    # Apply the net flows
    for city in food_flows:
        G.nodes[city]["resources"]["food"] += food_flows[city]

def simulate_economy(G, steps=5):
    """
    Run a simple production & trade simulation for the specified number of steps.
    Print out the results each turn.
    """
    for step in range(1, steps+1):
        # 1. Production
        produce_food(G, production_amount=10)
        
        # 2. Trade
        trade_food(G, trade_threshold=100, max_trade_per_edge=20)
        
        # Print summary
        print(f"\nAfter Step {step}:")
        for city in sorted(G.nodes()):
            blocked = " (BLOCKED)" if G.nodes[city]["blocked"] else ""
            food_amount = G.nodes[city]["resources"]["food"]
            print(f"  {city}{blocked}: food={food_amount:.1f}")

### 8) Demonstration: Run the Simulation

We first run a few simulation steps under normal conditions, then block **Berlin** to simulate conflict.

In [ ]:
print("=== Initial Resource Distribution ===")
for city in sorted(G.nodes()):
    print(f"  {city}: {G.nodes[city]['resources']}")

print("\n=== Running Simulation (no conflict) ===")
simulate_economy(G, steps=3)

# Now block a city to simulate conflict
G.nodes["Berlin"]["blocked"] = True
print("\n=== Berlin is now BLOCKED ===")

simulate_economy(G, steps=3)

### 9) Visualization

Finally, we visualize the network on a world map, with blocked capitals highlighted in orange.

In [ ]:
try:
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
except Exception as e:
    print("Could not load 'naturalearth_lowres'. Falling back to GeoJSON from GitHub.")
    fallback_url = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson"
    world = gpd.read_file(fallback_url)

capitals_gdf = gpd.GeoDataFrame(
    capitals_df,
    geometry=[Point(xy) for xy in zip(capitals_df.Longitude, capitals_df.Latitude)],
    crs="EPSG:4326"  # WGS84 lat/lon
)

fig, ax = plt.subplots(figsize=(12, 8))
world.plot(ax=ax, color='lightgray', edgecolor='white')

# Plot edges
for (cityA, cityB) in G.edges():
    (lonA, latA) = G.nodes[cityA]["pos"]
    (lonB, latB) = G.nodes[cityB]["pos"]
    
    line_geom = LineString([(lonA, latA), (lonB, latB)])
    gpd.GeoSeries([line_geom], crs="EPSG:4326").plot(
        ax=ax,
        color="blue",
        linewidth=1,
        alpha=0.5
    )

# Plot capitals as red circles; highlight blocked capitals in a different color
blocked_cities = [city for city in G.nodes if G.nodes[city]["blocked"]]
unblocked_cities = [city for city in G.nodes if not G.nodes[city]["blocked"]]

unblocked_df = capitals_gdf[capitals_gdf["Capital"].isin(unblocked_cities)]
blocked_df = capitals_gdf[capitals_gdf["Capital"].isin(blocked_cities)]

unblocked_df.plot(ax=ax, color='red', markersize=50, label="Unblocked")
blocked_df.plot(ax=ax, color='orange', markersize=100, marker="X", label="Blocked")

# Add labels to each city
for idx, row in capitals_gdf.iterrows():
    plt.text(
        x=row.geometry.x + 1,
        y=row.geometry.y + 1,
        s=row["Capital"],
        fontsize=9
    )

plt.title("World Capitals Network (Simple Simulation)")
plt.legend()
plt.show()

print("\nDone! You have run a simple conflict/economy simulation and visualized the result.")

## How the Model Works

### Nodes
Each node corresponds to a capital city. Attributes include:
- **pos**: `(longitude, latitude)`, used for distance calculations and plotting.
- **country**: The country name (e.g., "France").
- **blocked**: A boolean indicating if the city is blocked due to conflict.
- **resources**: A dictionary that holds resource quantities. Currently, only "food" is tracked.

### Edges
Edges represent potential trade routes between cities. An edge is created only if two cities are within a specified `DISTANCE_THRESHOLD` (3,000 km in this example). Each edge has:
- **distance**: The great-circle distance between the two cities in kilometers.
- **capacity**: A sample "capacity" attribute. This can be used to model throughput limits (e.g., how much a route can carry per time step).

### Blocked Nodes
When a node is **blocked**:
- It does not produce food (or other resources).
- It does not send or receive any resources via trade.

This simulates, for instance, a siege or an interruption in supply lines due to conflict.

### Simulation Flow
1. **Production**: Each unblocked node produces a certain amount of "food."  
2. **Trade**: Surplus food is shared with neighbors that have a deficit, capped by certain limits.  
3. **Conflict**: If a node is set to `blocked=True`, it is omitted from production and trade.

## How to Extend the Model

### Add New Resources
- Extend the `resources` dictionary to track multiple resources (e.g., `"oil"`, `"gold"`, etc.).
- Modify the `produce_food` function or create additional functions (e.g., `produce_oil`) to handle the production of different resources.
- Update the `trade_food` function to handle multiple resources, possibly with separate thresholds and trade caps.

### Refine Production and Trade Logic
- Make production dependent on city-specific factors (population, technology level, random events, etc.).
- Incorporate supply-demand curves or pricing into trade decisions.
- Account for transportation costs or time delays along edges.

### Advanced Conflict Rules
- **Block edges** instead of (or in addition to) nodes. Some routes might be blocked while others remain open.
- **Partial capacity reduction**: A city might lose only part of its production capacity or trade capacity.
- **Spreading conflict**: Conflict could spread to neighbors based on certain probabilities, or be resolved after a period of time.

### Dynamic or Time-Varying Graph Edges
- Let edges appear or disappear over time (e.g., roads closed, new trade agreements, infrastructure built).
- Implement random shocks or events (storms, diplomatic disputes) that temporarily reduce capacity.

## Summary

This simple economic conflict model is a starting point for exploring how disruptions can affect an interconnected network of cities. It demonstrates:
- Creating a geographic network of cities (nodes) with trade routes (edges).
- Simulating production and surplus/deficit-based trade.
- Handling conflict by blocking cities (removing them from production and trade).
- Visualizing the network on a world map for analysis.

By extending the model with additional resources, more complex production and trade rules, and advanced conflict dynamics, you can explore a wide range of scenarios in an accessible, Python-based framework.